#1. Importing libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import shap
from catboost import CatBoostRegressor
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from scipy.spatial import cKDTree
from pyproj import CRS, Transformer
from pykrige.ok import OrdinaryKriging

#2. Converting to Longitude Latitude



In [ ]:
# Loading the data
data = pd.read_csv('UT Comp_Seq.csv')

#the following function uses the pyproj transformer function to take long lat and transform it to feet coordinate system
def convert_lonlat_to_feet(longitude, latitude, target_crs):
    """
    Converts longitude and latitude coordinates to feet coordinate system.

    Args:
        longitude (float or list): Longitude value(s) in decimal degrees.
        latitude (float or list): Latitude value(s) in decimal degrees.
        target_crs (str): Target coordinate reference system in feet.
                           Example: 'epsg:2263' for NAD83 / New York Long Island (ftUS)

    Returns:
        tuple: A tuple containing x and y coordinates in feet.
    """
    source_crs = CRS("epsg:4326")
    target_crs = CRS(target_crs)
    #main function
    transformer = Transformer.from_crs(source_crs, target_crs, always_xy=True)
    x, y = transformer.transform(longitude, latitude)
    return x, y


# Target CRS: NAD83 / New York Long Island (ftUS)
target_crs = 'epsg:2263'

data['X (feet)'], data['Y (feet)'] = zip(*data.apply(lambda row: convert_lonlat_to_feet(row['Longitude'], row['Latitude'], target_crs), axis=1))

#zeros the data for convenience
data['X (feet)'] = data['X (feet)'] - data['X (feet)'].min()
data['Y (feet)'] = data['Y (feet)'] - data['Y (feet)'].min()

#saves to csv, the process can be quite lengthy, this is to save progress
data.to_csv('UT Comp_Seq_Feet.csv', index=False)

#3. Neighbor Counting


In [ ]:
# Loading the data from cell 2
data = pd.read_csv('UT Comp_Seq_Feet.csv')

#TF formation is divided into these following formations based on depth, combining them enables a better model
data['Formation'] = data['Formation'].replace(['TF2', 'TF1', 'TF3', 'UTFH', 'TFSH', 'MTFH', 'TF2.5', 'TF4'], 'TFH')
#convert it to a datetime object
data['Date Fracd'] = pd.to_datetime(data['Date Fracd'], format='%m/%d/%Y')
coords = data[['X (feet)', 'Y (feet)']].values
#uses cKDTree function, a spatial algorithm for faster calculations
tree = cKDTree(coords)

# Calculate neighbor well count with formation weighting
data['Neighbors_Count_UpToDate'] = 0  # Initialize the new column

for i in range(len(data)):
    current_date = data.loc[i, 'Date Fracd']
    neighbors_indices = tree.query_ball_point(coords[i], r=1000)
    #Counts the wells around the datapoint at the time of drilling
    neighbors_before_date = data.loc[neighbors_indices].loc[data['Date Fracd'] <= current_date]

    # Apply formation weighting for well count
    well_count = 0
    cum_prod = 0
    for neighbor_index in neighbors_before_date.index:
        if neighbor_index != i:  # Exclude the current well
            if neighbors_before_date.loc[neighbor_index, 'Formation'] == data.loc[i, 'Formation']:
                well_count += 1
            elif neighbors_before_date.loc[neighbor_index, 'Formation'] == 'MBH/TFH':
                well_count += 0.5
            else:
                well_count += 0.25
    for neighbor_index in neighbors_before_date.index:
        if neighbor_index != i:  # Exclude the current well
            if neighbors_before_date.loc[neighbor_index, 'Formation'] == data.loc[i, 'Formation']:
                well_count += 1
            elif neighbors_before_date.loc[neighbor_index, 'Formation'] == 'MBH/TFH':
                well_count += 0.5
            else:
                well_count += 0.25

    data.loc[i, 'Neighbors_Count_UpToDate'] = well_count

#saves to csv, the process can be quite lengthy, this is to save progress
data.to_csv('UT_Comp_With_Neighbor_Wells_1000_Formation_Weighted_Count.csv')

#4. Bootstrapping Catboost Model

In [ ]:
# Loading data from cell 3
data = pd.read_csv('UT_Comp_With_Neighbor_Wells_1000_Formation_Weighted_Count.csv')

# Creating ratios
data['Lateral Length/Stages'] = data['Lateral Length']/data['Stages']
data['Proppant/Stages'] = data['Total Prop, lbs']/data['Stages']

#Drop Features
data = data.drop(columns=['Fluid Type from DI', 'Well Name', 'Best1 Mo BOPD', 'Best9 Mo BOPD' ,'Best3 Mo BOPD',\
                            'Best6 Mo BOPD', 'Best12 Mo BOPD', 'Range', 'Township ', \
                            'SPACING_CAPPED', 'Lateral Length', 'Section', 'Unnamed: 0'])
data = data.dropna()
print(data.columns)

#Turn Date Fracd into Year Fracd
data['Date Fracd'] = data['Date Fracd'].str[-4:]
data['Date Fracd'] = pd.to_numeric(data['Date Fracd'])

# Get a list of categorical columns
categorical_cols = data.select_dtypes(include=['object']).columns.tolist()
# Label Encoding
label_encoder = LabelEncoder()
for col in categorical_cols:
    data[col] = label_encoder.fit_transform(data[col])
num_cols = data.shape[1]

# Prepare train-test split
X = data.drop(columns=['12 month Cum Prod'])
y = data['12 month Cum Prod']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Number of bootstrap iterations
n_bootstrap = 1 #100 used for actual code, changed to 1 for demonstration

# Store predictions for uncertainty estimation
bootstrap_preds = np.zeros((n_bootstrap, len(X_test)))

# Train multiple models on resampled data
for i in range(n_bootstrap):
    X_resampled, y_resampled = resample(X_train, y_train, random_state=i)
    best_param = {'learning_rate': 0.22, 'l2_leaf_reg': 7, 'iterations': 450, 'depth': 9, 'border_count': 270}
    model = CatBoostRegressor(loss_function="RMSE", silent=True, **best_param)
    model.fit(X_resampled, y_resampled)

    bootstrap_preds[i] = model.predict(X_test)


column_names = [f'Realization {i+1}' for i in range(n_bootstrap)]
df_pred_intervals = pd.DataFrame(bootstrap_preds.T, columns=column_names)


# Add actual values to the DataFrame
df_pred_intervals['Actual Value'] = y_test.values
df_pred_intervals['X (feet)'] = data['X (feet)']
df_pred_intervals['Y (feet)'] = data['Y (feet)']

# Save results to csv
df_pred_intervals.to_csv('prediction_intervals.csv', index=False)

#5. Bootstrapping SHAP Values

In [ ]:
#loading data
data = pd.read_csv('UT_Comp_With_Neighbor_Wells_1000_Formation_Weighted_Count.csv')

#zeroing x and y feet
data['X (feet)'] = data['X (feet)'] - data['X (feet)'].min()
data['Y (feet)'] = data['Y (feet)'] - data['Y (feet)'].min()

#creating ratios
data['Lateral Length/Stages'] = data['Lateral Length']/data['Stages']
data['Proppant/Stages'] = data['Total Prop, lbs']/data['Stages']

#Drop Features
data = data.drop(columns=['Fluid Type from DI', 'Well Name', 'Best1 Mo BOPD', 'Best9 Mo BOPD' ,'Best3 Mo BOPD',\
                            'Best6 Mo BOPD', 'Best12 Mo BOPD', 'Range', 'Township ', \
                            'SPACING_CAPPED', 'Lateral Length', 'Section', 'Unnamed: 0'])
data = data.dropna()
print(data.columns)

#Turn Date Fracd into Year Fracd
data['Date Fracd'] = data['Date Fracd'].str[-4:]
data['Date Fracd'] = pd.to_numeric(data['Date Fracd'])

# Get a list of categorical columns
categorical_cols = data.select_dtypes(include=['object']).columns.tolist()
# Label Encoding
label_encoder = LabelEncoder()
for col in categorical_cols:
    data[col] = label_encoder.fit_transform(data[col])
num_cols = data.shape[1]

# Prepare train-test split
X = data.drop(columns=['12 month Cum Prod'])
y = data['12 month Cum Prod']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Number of bootstrap iterations
n_bootstrap = 100 #100 was used for actual code, 1 for demo, to not blow up nataly's computer

# Store predictions for uncertainty estimation
shap_bootstrap = np.zeros((n_bootstrap, len(X)))
shap_bootstrap_df = pd.DataFrame(shap_bootstrap.T)

# Train multiple models on resampled data
for i in range(n_bootstrap):
    #Leaves out 10% of the data
    X_resampled, y_resampled = resample(X_train, y_train, random_state=i)
    #Train the model
    best_param = {'learning_rate': 0.22, 'l2_leaf_reg': 7, 'iterations': 450, 'depth': 9, 'border_count': 270}
    model = CatBoostRegressor(loss_function="RMSE", silent=True, **best_param)
    model.fit(X_resampled, y_resampled)
    #Use SHAP to explain the model
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X)
    shap_values_df = pd.DataFrame(shap_values, columns=[f'SHAP_{col}' for col in X.columns])
    #Calculate the location metric
    shap_bootstrap_df[i] = shap_values_df['SHAP_X (feet)'] + shap_values_df['SHAP_Y (feet)'] + shap_values_df['SHAP_Formation'] + shap_values_df['SHAP_Longitude'] + shap_values_df['SHAP_Latitude']


for i in range(n_bootstrap):
  shap_bootstrap_df = shap_bootstrap_df.rename(columns={i: f'Shap Realization {i + 1}'})

# save results to csv
shap_bootstrap_df['X (feet)'] = data['X (feet)']
shap_bootstrap_df['Y (feet)'] = data['Y (feet)']
shap_bootstrap_df.to_csv('shap_intervals_location.csv', index=False)

#6. Kriging Function Definition
####a. transform_and_export
It first randomly samples from a dataframe with 100 SHAP realizations for each well and transforms the distribution into a normal distribution

####b. adopt_highest_neighbor_value
Since the resultant distribution from the SHAP is rather noisy, we apply a smoothing algorithm to help out the kriging. We found that using highest value as opposed to simply the average outputted better models

####c. grid_data
The data was split into grids and grid blocks with values in them were averaged out and made to be the value of the grid, this is to make a lower resolution grid for the kriging algorithm later

####d. kriging
This is the main kriging function, it calls adopt_highest_neighbor_value and grid_data and then backtransforms the results.

In [ ]:
def transform_and_export(data, random_seed=None):
    """
    Randomly samples from 100 columns for every row, transforms values to a normal distribution,
    and returns a dataframe with the original sampled distribution and the transformed distribution

    Args:
        data: The input DataFrame.
        output_file: The path to the output CSV file.
        random_seed: The random seed to use for sampling (optional).

    Returns:
        results_df: The output dataframe with the location of each point, the randomly sampled data and the transformed value
    """



    # Extract columns for sampling
    shap_columns = [f"Shap Realization {i}" for i in range(1, 101)]

    # Create empty lists to store results
    sampled_values = []
    transformed_values = []

    # Set random seed if provided
    if random_seed is not None:
        np.random.seed(random_seed)

    # Iterate over each row
    for index in data.index:
        # Randomly sample one column
        random_column = np.random.choice(shap_columns)

        # Get the value from the sampled column
        sampled_value = data.loc[index, random_column]

        # Append to the list of sampled values
        sampled_values.append(sampled_value)

    # Calculate mean and standard deviation of sampled values
    mean = np.mean(sampled_values)
    std = np.std(sampled_values)

    # Transform sampled values to normal distribution
    transformed_values = [(x - mean) / std for x in sampled_values]

    # Create a DataFrame for the results
    results_df = pd.DataFrame({
        "Sampled Value": sampled_values,
        "Transformed Value": transformed_values,
        "X (feet)": data["X (feet)"],
        "Y (feet)": data["Y (feet)"]
        })

    return results_df

def adopt_highest_neighbor_value(data, distance_threshold=8000):
    """
    Assigns the highest "Transformed Value" within a specified distance
    to each data point. Uses cKDTree for efficient neighbor search.

    Args:
        data: The input DataFrame with "X (feet)", "Y (feet)", and "Transformed Value" columns.
        distance_threshold: The maximum distance (in feet) to consider neighbors.

    Returns:
        A new DataFrame with the updated "Transformed Value" column.
    """
    modified_data = data.copy()
    coords = modified_data[["X (feet)", "Y (feet)"]].values
    values = modified_data["Transformed Value"].values

    # Build a cKDTree for efficient neighbor search
    tree = cKDTree(coords)

    for i in range(len(coords)):
        # Find neighbors within the distance threshold
        neighbors_indices = tree.query_ball_point(coords[i], distance_threshold)

        # Exclude the current point from neighbors
        neighbors_indices = [j for j in neighbors_indices if j != i]

        # Get values of neighbors
        neighbor_values = values[neighbors_indices]

        # Assign the highest value (handle case with no neighbors)
        if len(neighbor_values) > 0:
            highest_value = np.max(neighbor_values)
        else:
            highest_value = values[i]  # Keep original value if no neighbors

        # Update the "Transformed Value" for the current point
        modified_data.loc[i, "Transformed Value"] = highest_value

    return modified_data

def grid_data(data, grid_size):
    """
    Transforms data into a grid, averaging values within each grid cell.
    """

    # Get data bounds and define grid cell size
    xmin, xmax = data["X (feet)"].min(), data["X (feet)"].max()
    ymin, ymax = data["Y (feet)"].min(), data["Y (feet)"].max()
    cell_size_x = (xmax - xmin) / grid_size
    cell_size_y = (ymax - ymin) / grid_size

    # Create grid centers
    x_centers = np.linspace(xmin + cell_size_x / 2, xmax - cell_size_x / 2, grid_size)
    y_centers = np.linspace(ymin + cell_size_y / 2, ymax - cell_size_y / 2, grid_size)

    # Create meshgrid
    grid_x_coords, grid_y_coords = np.meshgrid(x_centers, y_centers)

    # Create empty grid to store averaged values
    grid = np.full((grid_size, grid_size), np.nan)  # Initialize with NaN

    # Assign averaged values to grid cells
    for i in range(grid_size):
        for j in range(grid_size):
            # Find data points within the current grid cell
            cell_mask = (
                (data["X (feet)"] >= x_centers[i] - cell_size_x / 2)
                & (data["X (feet)"] < x_centers[i] + cell_size_x / 2)
                & (data["Y (feet)"] >= y_centers[j] - cell_size_y / 2)
                & (data["Y (feet)"] < y_centers[j] + cell_size_y / 2)
            )

            # Calculate average value for the cell (if any data points are found)
            if cell_mask.any():
                grid[j, i] = data.loc[cell_mask, "Transformed Value"].mean()

    return grid, x_centers, y_centers, grid_x_coords, grid_y_coords

def kriging(data, grid_size):
    """Performs Ordinary Kriging and back-transforms the results."""

    original_mean = data["Sampled Value"].mean()
    original_std = data["Sampled Value"].std()
    data = adopt_highest_neighbor_value(data)
    grid, x_centers, y_centers, grid_x_coords, grid_y_coords = grid_data(data, grid_size)

    # Extract valid grid points
    valid_mask = ~np.isnan(grid)
    x_known = grid_x_coords[valid_mask]
    y_known = grid_y_coords[valid_mask]
    z_known = grid[valid_mask]

    # Perform Ordinary Kriging
    OK = OrdinaryKriging(x_known, y_known, z_known, variogram_model="spherical", verbose=False, enable_plotting=False)
    z_interp, ss = OK.execute("grid", x_centers, y_centers)
    ss = ss*original_std + original_mean

    # Back-transform and return
    z_interp_back = z_interp * original_std + original_mean
    return x_centers, y_centers, z_interp_back, x_known, y_known, z_known, ss

#7. Main Kriging Loop
creates multiple kriging realizations and displays the average kriging value, the average variance, and the interval widths

In [ ]:
# Load the data from cell 5
data = pd.read_csv('shap_intervals_location.csv')
num_realizations = 50 #number of kriging
# Store kriging results, known data points, and variance for averaging
all_z_interp_back = []
x_known = []  # Store x_known only once
y_known = []  # Store y_known only once
all_z_known_back = []
all_ss = []  # Store kriging variance (ss)
all_kriged_results = []  # Store kriged results for each realization

#Main loop
for i in range(num_realizations):
    sampled_data = transform_and_export(data, random_seed=i)
    x_centers, y_centers, z_interp_back, x_known_current, y_known_current, z_known, ss = kriging(sampled_data, grid_size=150)

    if i == 0:  # Store x_known and y_known only once
        x_known = x_known_current
        y_known = y_known_current

    all_z_interp_back.append(z_interp_back)
    all_z_known_back.append(z_known)
    all_ss.append(ss)

    # Store kriged results for this realization
    kriged_results = pd.DataFrame({
        "X (feet)": np.tile(x_centers, len(y_centers)),  # Repeat x_centers for each y_center
        "Y (feet)": np.repeat(y_centers, len(x_centers)),  # Repeat y_centers for each x_center
        "Kriged Value": z_interp_back.flatten()  # Flatten the kriged values
    })
    all_kriged_results.append(kriged_results)

# Calculate averages
average_z_interp_back = np.mean(all_z_interp_back, axis=0)
average_z_known_back = np.mean(all_z_known_back, axis=0)
average_ss = np.mean(all_ss, axis=0)

# --- Plotting ---
plt.figure(figsize=(15, 15))
plt.contourf(x_centers, y_centers, average_z_interp_back, levels=500, cmap="inferno")
plt.colorbar(label="Average Back-Transformed Kriged Value")

# Scatter plot of average known data points
plt.scatter(x_known, y_known, c=average_z_known_back,
            edgecolors="black", label="Average Original Grid Cells", cmap="inferno")

plt.title("Average Kriging Interpolation (Back-Transformed to Original Scale)")
plt.xlabel("X (feet)")
plt.ylabel("Y (feet)")
plt.legend()
plt.tight_layout()
plt.show()

# 2. Plot Kriging Variance
plt.figure(figsize=(15, 15))
plt.contourf(x_centers, y_centers, average_ss, levels=500, cmap="viridis")  # Use a different colormap
plt.colorbar(label="Average Kriging Variance")
plt.title("Kriging Variance")
plt.xlabel("X (feet)")
plt.ylabel("Y (feet)")
plt.tight_layout()
plt.show()

all_kriged_results_df = pd.concat(all_kriged_results)

# Calculate interval widths for each location
location_stats = all_kriged_results_df.groupby(["X (feet)", "Y (feet)"])["Kriged Value"].agg(["min", "max"])
location_stats["Interval Width"] = location_stats["max"] - location_stats["min"]

# Reshape interval width for plotting
interval_width_grid = location_stats["Interval Width"].values.reshape(len(y_centers), len(x_centers))

# === Plot the interval width grid ===
plt.figure(figsize=(15, 15))
plt.contourf(x_centers, y_centers, interval_width_grid, levels=500, cmap="viridis")
plt.colorbar(label="Interval Width")
plt.title("Kriging Interval Width")
plt.xlabel("X (feet)")
plt.ylabel("Y (feet)")
plt.tight_layout()
plt.show()